In [ ]:
import os
import zipfile
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_zip_path = "/content/drive/MyDrive/Prodigy/archive (7).zip"
data_extract_path = "/content/LeapGestRecog"

with zipfile.ZipFile(data_zip_path, 'r') as zip_file:
    zip_file.extractall(data_extract_path)


image_transforms = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


dataset_path = '/content/LeapGestRecog/leapGestRecog'
dataset = ImageFolder(root=dataset_path, transform=image_transforms)


train_ratio = 0.8
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size

train_data, test_data = random_split(dataset, [train_size, test_size])

In [ ]:
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)


class HandGestureModel(nn.Module):
    def __init__(self, num_classes):
        super(HandGestureModel, self).__init__()

        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [ ]:
num_classes = len(dataset.classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = HandGestureModel(num_classes).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(model, dataloader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_images, batch_labels in dataloader:
            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)

            optimizer.zero_grad()

            predictions = model(batch_images)
            loss = criterion(predictions, batch_labels)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}")


num_epochs = 10
train(model, train_loader, loss_function, optimizer, num_epochs)

Epoch [1/10], Loss: 0.5887
Epoch [2/10], Loss: 0.0606
Epoch [3/10], Loss: 0.0329
Epoch [4/10], Loss: 0.0276
Epoch [5/10], Loss: 0.0382
Epoch [6/10], Loss: 0.0191
Epoch [7/10], Loss: 0.0310
Epoch [8/10], Loss: 0.0264
Epoch [9/10], Loss: 0.0182
Epoch [10/10], Loss: 0.0232


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for batch_images, batch_labels in dataloader:
            batch_images, batch_labels = batch_images.to(device), batch_labels.to(device)
            predictions = model(batch_images)
            _, predicted_labels = torch.max(predictions, 1)

            total_samples += batch_labels.size(0)
            correct_predictions += (predicted_labels == batch_labels).sum().item()

    accuracy = (correct_predictions / total_samples) * 100
    print(f"Test Accuracy: {accuracy:.2f}%")


In [ ]:
evaluate(model, test_loader)

Test Accuracy: 99.02%
